In [1]:
# Import packages
library(tidyverse)
library(bnlearn)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [4]:
met_source = "metno"
end_yr_li = list('metno'=2018,
                 'era5'=2018,
                 'nomet'=2020)

# Fitted bnlearn objects
rfile_fpath = sprintf("../Data/RData/Vansjo_fitted_GaussianBN_%s_1981-%s.rds", met_source, end_yr_li[[met_source]])

out_folder = "../Data/FittedNetworkDiagnostics"

# Pre-calculated standard deviations (from fitting of GBN in NB 02)
sd_fpath = file.path(out_folder, sprintf("GBN_%s_1981-%s_stdevs.csv", met_source, end_yr_li[[met_source]]))

In [8]:

year = 2000
chla_prevSummer = 2
colour_prevSummer = 10
TP_prevSummer = 0.01
wind_speed = 3
rain = 200


suppressMessages(library(tidyverse))
suppressMessages(library(bnlearn))    

# Build dataframe from input data
year <- c(year)
chla_prevSummer <- c(chla_prevSummer)
colour_prevSummer <- c(colour_prevSummer)
TP_prevSummer <- c(TP_prevSummer)
wind_speed <- c(wind_speed)
rain <- c(rain)

driving_data <- data.frame(chla_prevSummer, colour_prevSummer, TP_prevSummer, wind_speed, rain)
row.names(driving_data) <- year

# Convert any integer cols to numeric
driving_data[1:ncol(driving_data)] = lapply(driving_data[1:ncol(driving_data)], as.numeric)

# Load fitted Bayesian network and pre-calculated std. devs.
fitted_BN = readRDS(rfile_fpath)
sds = read.csv(file=sd_fpath, header=TRUE, sep=",")

# Standard deviations: just select values associated with nodes for prediction, sort alphabetically
sd_predictedNodes = filter(sds, node %in% nodes_to_predict)
sd_predictedNodes = sd_predictedNodes[order(sd_predictedNodes$node),]

# Nodes to make predictions for. Must match nodes present in the fitted BN.
# Add check that list is sorted alphabetically, as concatenation of final df assumes this
nodes_to_predict = sort(c('chla','colour','cyano', 'TP'))

sd_predictedNodes

,node,sd
,<chr>,<dbl>
2,chla,3.7389697
4,colour,8.9597100
3,cyano,0.7381779
1,TP,3.7958656


In [29]:
bias_adj = FALSE

set.seed(1)

# Loop over nodes and derive expected values
expectedValue_li = vector(mode = "list", length = 0)

for (cur_node in nodes_to_predict) {
    pred = predict(fitted_BN,
                   data=driving_data,
                   node=cur_node,
                   method='bayes-lw',
                   n=10000
                  )
    sigma = subset(sd_predictedNodes, node == "TP")$sd

    # If node is cyano, then remove the boxcox transformation before adding expected value to list
    # 0.1 is chosen lambda value (specific to this data)
    if (cur_node == "cyano") {
        if (bias_adj == TRUE) {
            pred = (pred*0.1 + 1)**(1/0.1) * (1 + ((sigma**2 * (1-0.1))/
                                              (2*(0.1*pred+1)**2)))
        } else {
            # Back transform without bias adjustment.
            pred = (pred*0.1 + 1)**(1/0.1)
        }
    }
    expectedValue_li[[cur_node]] = pred # Update list with value for this node
}

# Sort alphabetically
expectedValue_li = expectedValue_li[order(names(expectedValue_li))]

expectedValue_li

$chla
[1] 5.427805

$colour
[1] 6.045492

$cyano
[1] 0.3317043

$TP
[1] 10.73924

In [ ]:
# Thresholds to use in classification
boundaries_list = list('TP' = 29.5,     # Middle of 'Moderate' class
                       'chla' = 20.0,   # M-P boundary. WFD boundaries: [10.5, 20.0]. Only 6 observed points under 10.5 so merge G & M
                       'colour' = 48.0, # 66th percentile (i.e. upper tercile). No management implications
                       'cyano' = 1.0    # M-P boundary is 2.0, but there were only 2 values in this class. Plenty above 2 tho
                       )

# Sort alphabetically
boundaries_list = boundaries_list[order(names(boundaries_list))] 

# Data for evidence, converted to named list
evidence_li = as.list(driving_data)

# Empty list to be populated with probability of being below boundary
prob_li = vector(mode = "list", length = 0)

# Loop over nodes and predict probabilities of being within various classes
for (node in nodes_to_predict){
    boundary = unlist(boundaries_list[node], use.names = FALSE)

    # If cyanomax, apply boxcox transformation with lambda=0.1 to boundary first
    if (node == 'cyano') {
        boundary = (boundary^0.1 - 1) / 0.1
    }

    # Horrible hack to make 'query' available in the global namespace. See
    #     https://stackoverflow.com/q/19260580/505698
    query <<- paste("(", node, ' < ', boundary, ")", sep="")
    prob = cpquery(fitted_BN,
                   event = eval(parse(text = query)),
                   evidence = evidence_li,
                   method = 'lw'
                  )

    # Round to 2 d.p. Below this, cpquery returns variable results over diff calls
    # Even with rounding, still get some variability in results
    prob = round(prob, digits = 2)

    prob_li[[node]] = prob
}

# Sort alphabetically
prob_li = prob_li[order(names(prob_li))] # Sort alphabetically

# Build dataframe
prob_df = data.frame(node=nodes_to_predict,
                     threshold = unlist(boundaries_list, use.names=FALSE),
                     prob_below_threshold = unlist(prob_li, use.names=FALSE),
                     prob_above_threshold = 1-unlist(prob_li, use.names=FALSE),
                     expected_value = signif(unlist(expectedValue_li, use.names=FALSE),3), #Round to 3 s.f
                     st_dev = signif(sd_predictedNodes['sd'], 3)                           # Round to 3 s.f
                    )

return(prob_df)


In [2]:
library(bnlearn)
dag = hc(gaussian.test)
fitted = bn.fit(dag, gaussian.test)
fitted


  Bayesian network parameters

  Parameters of node A (Gaussian distribution)

Conditional density: A
Coefficients:
(Intercept)  
   1.007493  
Standard deviation of the residuals: 1.004233 

  Parameters of node B (Gaussian distribution)

Conditional density: B
Coefficients:
(Intercept)  
   2.039499  
Standard deviation of the residuals: 3.034111 

  Parameters of node C (Gaussian distribution)

Conditional density: C | A + B
Coefficients:
(Intercept)            A            B  
   2.001083     1.995901     1.999108  
Standard deviation of the residuals: 0.5089772 

  Parameters of node D (Gaussian distribution)

Conditional density: D | B
Coefficients:
(Intercept)            B  
   5.995036     1.498395  
Standard deviation of the residuals: 0.3286672 

  Parameters of node E (Gaussian distribution)

Conditional density: E
Coefficients:
(Intercept)  
   3.493906  
Standard deviation of the residuals: 1.98986 

  Parameters of node F (Gaussian distribution)

Conditional density: F |

In [3]:
lapply(as.lm(fitted, gaussian.test), confint)

,2.5 %,97.5 %
(Intercept),0.9796513,1.035336
,2.5 %,97.5 %
(Intercept),1.955378,2.123619
,2.5 %,97.5 %
(Intercept),1.978794,2.023372
A,1.981843,2.009959
B,1.994455,2.003761
,2.5 %,97.5 %
(Intercept),5.984056,6.006015
B,1.495392,1.501399


In [4]:
gaussian.test

A,B,C,D,E,F,G
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1.1130829,1.93216393,7.07480586,8.660411,0.8815912,24.71950,9.216381507
-0.2479482,11.33434292,24.34737157,23.355432,7.0401132,36.81300,3.678833044
1.8545084,3.03202012,11.08647349,11.055891,3.8345302,22.01718,2.424513357
0.8339109,3.85797013,11.22477498,11.937471,1.0056242,23.28564,6.085472840
0.4886137,4.51261310,10.00047640,12.537179,4.0847458,24.53760,5.117575201
1.6495578,0.85081688,6.85705964,7.094566,6.5108420,27.69884,7.691891660
1.3568921,1.90603085,8.54754019,8.777001,7.0103860,24.88071,4.376690771
0.7916755,-1.25669757,-0.06165925,4.000863,5.9748497,17.20239,3.767514789
1.8955701,5.07236064,15.08856806,14.044994,5.8411179,30.26592,3.557995696
